# Assignment 2.

## Formalia:

Please read the [assignment overview page](https://github.com/suneman/socialdata2021/wiki/Assignment-1-and-2) carefully before proceeding. This page contains information about formatting (including formats etc), group sizes, and many other aspects of handing in the assignment. 

_If you fail to follow these simple instructions, it will negatively impact your grade!_

**Due date and time**: The assignment is due on Monday April 5th, 2021 at 23:55. Hand in your files via [`http://peergrade.io`](http://peergrade.io/).

**Peergrading date and time**: _Remember that after handing in you have a week to evaluate a few assignments written by other members of the class_. Thus, the peer evaluations are due on Monday April 12th, 2021 at 23:55. 

## Part 1: Questions to text and lectures.

A) Please answer my questions to the Segal and Heer paper we read during lecture 7 and 8.

* What is the *Oxford English Dictionary's* defintion of a narrative?
* What is your favorite visualization among the examples in section 3? Explain why in a few words.
* What's the point of Figure 7?
* Use Figure 7 to find the most common design choice within each category for the Visual narrative and Narrative structure (the categories within visual narrative are 'visual structuring', 'highlighting', etc).
* Check out Figure 8 and section 4.3. What is your favorite genre of narrative visualization? Why? What is your least favorite genre? Why?


B) Also please answer the questions to my talk on [explanatory data visualization](https://www.youtube.com/watch?v=yHKYMGwefso)

* What are the three key elements to keep in mind when you design an explanatory visualization?
* In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
  - Go online and find a visualization that follows these principles (don't use one from the video). 
  - Explain how it does achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
* Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

## Part 2: Random forest and weather

The aim here is to recreate the work you did in Part 1-3 of the Week 7 lecture. I've phrased things differently relative to the exercise to make the purpose more clear. 

Part 2A: Random forest binary classification. 
* Using the and instructions and material from Week 7, build a *random forest* classifier to distinguish between two types (you choose) of crime using on spatio-temporal (where/when) features of data describing the two crimes. When you're done, you should be able to give the classifier a place and a time, and it should tell you which of the two  types of crime happened there.
  - Explain about your choices for training/test data, features, and encoding. (You decide how to present your results, but here are some example topics to consider: Did you balance the training data? What are the pros/cons of balancing? Do you think your model is overfitting? Did you choose to do cross-validation? Which specific features did you end up using? Why? Which features (if any) did you one-hot encode? Why ... or why not?))
  - Report accuracy. Discuss the model performance.
  
  
Part 2B: Info from weather features.
* Now add features from weather data to your random forest. 
  - Report accuracy. 
  - Discuss how the model performance changes relative to the version with no weather data.
  - Discuss what you have learned about crime from including weather data in your model.

In [31]:
#Loading data
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\andri\Documents\Andri Geir\DTU\Social Data\Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")

df= df[(pd.to_datetime(df['Date']) >= '01/01/2003')] 
df = df[(pd.to_datetime(df['Date']) <= '31/12/2017')] 
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Hour'] = [int(time[0:2]) for time in (df['Time'])]

## Creating balanced dataset
df_filtered1 = df[df["Category"]==("VEHICLE THEFT")].sample(40000)
df_filtered2 = df[df["Category"]==("FRAUD")].sample(40000)
df_balanced = pd.concat([df_filtered1,df_filtered2]).sample(frac=1).reset_index()

# One hot Encoding and factorizing the categorical target variable.
X = pd.get_dummies(df_balanced[["PdDistrict","Year","Month","Day","Hour"]])
y = df_balanced["Category"]
y = y.factorize( ['VEHICLE THEFT', 'FRAUD'] )[0]


# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25)

##### Loading and test/train split
To begin with the data is loaded, filtered for dates from 2003-2018 and then "Month of the year" and "Time of the day" columns added according to instructions. The we draw equal amounts of observations from each category in order to maintain balance in the data set Having such a balanced data set ensures that the model is gets euqal amount of each class but it also "deletes" the information that one category is more common than the other. This information could of course greatly improve such a model in cases where the model maybe isn't very . The dataset is also shuffled so that train and test draw later isn't biast.

Before we split into the train and test the categorical varialbe "PdDistrict" is one hot encoded because the model can not understand text so that variable is split into multible and represented as binary one for each district. The target variable which is categorical with only two classes is factorized to be either 0 or 1, again so the model can understand it.

Finally the data set is split into 75% train data and 25% test.

In [32]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)

error  = abs(test_labels-predictions.round()).sum()
(len(test_labels)-error)/len(test_labels)




0.682

###### Accuracy 
The model predicts 68% correct which is better than guessing but still not very impressive.

In [33]:

#Prediction for one observation
rf.predict(np.array(test_features.iloc[0,:]).reshape(1, -1))

array([0.4855])

If one want to make a prediction for a new observation the above code is an example of this. After rounding this will indictate  1 or 0 for either predicting the crime to be a Vehicle Theft or Fraud.

In [34]:
# REading dat and localizing the time after instructions on slack.
weather = pd.read_csv("weather_data.csv", parse_dates=["date"],
                date_parser=lambda x: pd.to_datetime(x).tz_convert(None).tz_localize("Etc/GMT+3").tz_convert("Etc/GMT-7")) 
# parse_dates specifies what columns contain dates (instead of a string column -> it becomes a date_time column)
# data_parser -> we specify our custom date_parser (Pandas has default data_parser, usually we do not need to specify it)
# in our data_parser we use "lambda" function - it means that we want to apply something to each value in the column
# pd.to_datetime(x) - converts each value to date_time obect. By default pd.to_datetime assigns GMT0 timezone, 
# which is wrong, thus, we specification of timezone with tz_convert(None)
# now we want to specify the correct timezone -> we use tz_localize("..")
# after we can convert dates to the actual SanFrancisco timezone with tz_convert("..")
# weather.head()


#Loading and adding relevant columns again
df = pd.read_csv(r"C:\Users\andri\Documents\Andri Geir\DTU\Social Data\Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv", usecols=["Category", "Date", "Time", "PdDistrict"]) ## specify any columns you need

df = df[df["Category"].isin(['VEHICLE THEFT', 'FRAUD'])] # filter out the dataframe, you can plug any list of crimes
df["datetime"] = df.apply(lambda x: pd.to_datetime(x.Date + " " + x.Time).round("H").tz_localize("ETC/GMT-7"), axis = 1)  
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Hour'] = [int(time[0:2]) for time in (df['Time'])]
# Here we do a bit more complicated thing
# .apply allows us to use function for each row of a dataframe (read documentation for more info)
# so we take a row (which is x) and take cell of Date and Time -> and concatenate them to one big string
# that can be then converted to datetime. We would also want to remove any seconds and minutes (round to hours)
# then we specify that dates are in GMT-7
# the result is going to be stored in new "datetime" column


# Left joining in by time stamp in order to get the relevant weather data for each observation.
df_weather = pd.merge(df,weather,how="left",left_on ="datetime",right_on ="date")
df_weather = df_weather.dropna(subset=["date"])

#Balancing the data after the observatoions with the wather data missing have been dropped.
df_filtered1 = df_weather[df_weather["Category"]==("VEHICLE THEFT")].sample(14301)
df_filtered2 = df_weather[df_weather["Category"]==("FRAUD")].sample(14301)

#Constructing the final df to work with.
df_balanced = pd.concat([df_filtered1,df_filtered2]).sample(frac=1).reset_index()



X = pd.get_dummies(df_balanced[['PdDistrict', 'Year',
       'Month', 'Day', 'Hour', 'temperature', 'humidity', 'weather',
       'wind_speed', 'wind_direction', 'pressure']])

y = df_balanced["Category"]
y = y.factorize( ['VEHICLE THEFT', 'FRAUD'] )[0]

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.25, random_state = 42)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
#print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
error  = abs(test_labels-predictions.round()).sum()
(len(test_labels)-error)/len(test_labels)

0.6688575024472102

The procedure is exactly like before and the same variables are one hot encoded and factorized. All of the weather data is continous so no need to manipulate that. 

The model

## Part 3: Data visualization

* Create the Bokeh visualization from Part 2 of the Week 8 Lecture, displayed in a beautiful `.gif` below. 
* Provide nice comments for your code. Don't just use the `# inline comments`, but the full Notebook markdown capabilities and explain what you're doing.

![Movie](https://github.com/suneman/socialdataanalysis2020/blob/master/files/week8_1.gif?raw=true "movie")